In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import xml.etree.ElementTree as ET
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.preprocessing.image import img_to_array, load_img
from keras.utils import to_categorical

In [ ]:
# Path to the directory containing images and annotations
data_path = 'drive/MyDrive/ColabNotebooks/street-sign-data'

# Loading Images (PNG) and Annotations (XML)
image_folder = os.path.join(data_path, 'images')
annotation_folder = os.path.join(data_path, 'annotations')

# Lists to store features (X) and labels (y)
X = []
y = []

# Iterating over image files
for image_file in os.listdir(image_folder):
    # Load Image
    image_path = os.path.join(image_folder, image_file)
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)
    #image_array = preprocess_input(image_array)

    # Using the image as a feature
    X.append(image_array)

    # Extracting corresponding label from the XML annotation
    annotation_file = os.path.join(annotation_folder, image_file.replace('.png', '.xml'))
    tree = ET.parse(annotation_file)
    root = tree.getroot()
    class_label = root.find('.//object/name').text
    if class_label == 'speedlimit':
        class_label = 0
    elif class_label == 'stop':
        class_label = 1
    elif class_label == 'crosswalk':
        class_label = 2
    elif class_label == 'trafficlight':
        class_label = 3
    y.append(class_label)


# Converting lists to numpy arrays
X = np.array(X)
y = np.array(y)

pre_processed_X = []

for image in X:
  pre_processed_image = keras.applications.vgg16.preprocess_input(image)
  pre_processed_X.append(pre_processed_image)

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_classes = len(np.unique(y_train))
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

In [ ]:
print("image size: ", X_train[100].shape)
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

#y_train = y_train.reshape(701, 7, 7, 512)
#print(f"y_train shape: {y_train.shape}")

In [ ]:
#load pre-trained model
model = VGG16(weights='imagenet', include_top=False, classes = 4)
x = model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(4, activation='softmax')(x)  # Output layer

model = Model(inputs=model.input, outputs=predictions)

"""for layer in model.layers:
    layer.trainable = False"""

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=10, batch_size=32)

accuracy = model.evaluate(X_test, y_test_encoded)[1]
print(f"Accuracy: {accuracy}")